# Bases de datos con SQLAlchemy y SQLite

In [1]:
import sqlite3
from sqlalchemy import create_engine, Table, MetaData, select, or_, and_, func

## Join entre tablas de la misma base de datos

In [2]:
# Creamos una base de datos y un cursor
conn_sqlite_biblioteca = sqlite3.connect('./biblioteca.db')
cursor = conn_sqlite_biblioteca.cursor()

In [3]:
# Creamos una tabla
cursor.execute("""CREATE TABLE escritor (
                    'id' INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
                    'nombre' TEXT NOT NULL,
                    'paterno' TEXT NOT NULL,
                    'materno' TEXT DEFAULT " ",
                    'profesion' TEXT DEFAULT "escritor");""")
conn_sqlite_biblioteca.commit()

In [4]:
# Verificamos la creación de la tabla
engine = create_engine('sqlite:///biblioteca.db')
conn_sqlalchemy = engine.connect()
metadata = MetaData()
escritor = Table('escritor', metadata, autoload=True, autoload_with=engine)
print(escritor.columns.keys())

['id', 'nombre', 'paterno', 'materno', 'profesion']


In [5]:
# Introducimos algunos valores
cursor.execute("""INSERT INTO escritor (nombre, paterno) 
                   VALUES 
                   ('Hermann','Hesse'),
                   ('Jostein', 'Gaarder');""")
cursor.execute("""INSERT INTO escritor (nombre, paterno, materno)
                    VALUES
                    ('Jorge', 'Volpi', 'Escalante');""")
conn_sqlite_biblioteca.commit()

In [6]:
# Hacemos una consulta: el apellido paterno de aquellos escritores cuyo nombre sea Hermann o cuyo id sea 3
query = select([escritor.columns.paterno])
query = query.where(
                or_(escritor.columns.nombre == 'Hermann', 
                    escritor.columns.id == 3))
results_proxy = conn_sqlalchemy.execute(query)
results = results_proxy.fetchall()
for result in results:
    print(result)

('Hesse',)
('Volpi',)


In [7]:
# Creamos otra tabla
cursor.execute("""CREATE TABLE libro (
                    'id' INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
                    'titulo' TEXT NOT NULL,
                    'descripcion' TEXT NOT NULL DEFAULT " ",
                    'autor' INTEGER NOT NULL,
                    FOREIGN KEY(autor) REFERENCES escritor(id));""")
conn_sqlite_biblioteca.commit()

In [8]:
# Verificamos la creación de la tabla
libro = Table('libro', metadata, autoload=True, autoload_with=engine)
print(libro.columns.keys())

['id', 'titulo', 'descripcion', 'autor']


In [9]:
# Introducimos algunos valores
cursor.execute("""INSERT INTO libro (titulo, autor) 
                   VALUES 
                   ('Demian',1),
                   ('Siddhartha', 1),
                   ('El lobo estepario',1);""")
cursor.execute("""INSERT INTO libro (titulo, autor, descripcion)
                    VALUES
                    ('El Mundo de Sofia', 2, 'Novela sobre la historia de la filosofía');""")
conn_sqlite_biblioteca.commit()

In [10]:
# Hacemos otra consulta: la descripción de los libros cuyo titulo empiece con E
query = select([libro.columns.descripcion])
query = query.where(libro.columns.titulo.like('E%'))
results_proxy = conn_sqlalchemy.execute(query)
results = results_proxy.fetchall()
for result in results:
    print(result)

(' ',)
('Novela sobre la historia de la filosofía',)


In [11]:
# Hacemos un inner join entre las tablas y nos quedamos con id del libro, titulo y nombre del autor
query = select([libro.columns.id, libro.columns.titulo, escritor.columns.nombre])
query = query.select_from(escritor.join(libro, escritor.columns.id == libro.columns.autor))
results_proxy = conn_sqlalchemy.execute(query)
results = results_proxy.fetchall()
for result in results:
    print(result)

(1, 'Demian', 'Hermann')
(2, 'Siddhartha', 'Hermann')
(3, 'El lobo estepario', 'Hermann')
(4, 'El Mundo de Sofia', 'Jostein')


In [12]:
# ¿Cuantos libros tiene registrado cada autor?
query = select([escritor.columns.nombre, func.sum(1)])
query = query.select_from(escritor.join(libro, escritor.columns.id == libro.columns.autor))
query = query.group_by(escritor.columns.nombre)
results_proxy = conn_sqlalchemy.execute(query)
results = results_proxy.fetchall()
for result in results:
    print(result)

('Hermann', 3)
('Jostein', 1)


## Join entre dos tablas de distintas bases

In [13]:
# Creamos otra base de datos
conn_sqlite_ciudad = sqlite3.connect('./ciudad.db')
cursor = conn_sqlite_ciudad.cursor()

In [14]:
# Creamos una tabla
cursor.execute("""CREATE TABLE ciudadano (
                    'id' INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
                    'nombre' TEXT NOT NULL,
                    'paterno' TEXT NOT NULL,
                    'materno' TEXT DEFAULT " ",
                    'profesion' TEXT DEFAULT " ");""")
conn_sqlite_ciudad.commit()

In [15]:
# Verificamos la creación de la tabla
engine = create_engine('sqlite:///ciudad.db')
conn_sqlalchemy = engine.connect()
metadata = MetaData()
ciudadano = Table('ciudadano', metadata, autoload=True, autoload_with=engine)
print(ciudadano.columns.keys())

['id', 'nombre', 'paterno', 'materno', 'profesion']


In [16]:
# Introducimos algunos valores
cursor.execute("""INSERT INTO ciudadano (nombre, paterno, profesion) 
                   VALUES 
                   ('Hermann', 'Hesse', 'escritor'),
                   ('Miguel', 'Alcubierre', 'físico');""")
cursor.execute("""INSERT INTO ciudadano (nombre, paterno, materno)
                    VALUES
                    ('Juan', 'Hernández', 'Pérez');""")
conn_sqlite_ciudad.commit()

In [17]:
# Hacemos una consulta: el nombre de las personas sin profesion registrada
query = select([ciudadano.columns.nombre])
query = query.where(ciudadano.columns.profesion == " ")
results_proxy = conn_sqlalchemy.execute(query)
results = results_proxy.fetchall()
for result in results:
    print(result)

('Juan',)


In [73]:
engine = create_engine('sqlite:///')
conn_sqlalchemy = engine.connect()
metadata = MetaData(schema='biblioteca.db')
tabla = Table('libro', metadata, schema='biblioteca.db')
#query = select([ciudadano])
#results_proxy = conn_sqlalchemy.execute(query)
#results = results_proxy.fetchall()
#for result in results:
#    print(result)
print(tabla.columns.keys())

[]
